In [2]:
import requests

In [5]:
game_data = []
r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/2019020005/feed/live')
data = r.json()
game_data.append(data)
plays = data['liveData']['plays']['allPlays'] 

In [10]:
event_types = ['Shot','Blocked Shot','Missed Shot']
shot_data = {}
shot_data['shotType'] = []
shot_data['team'] = []

In [12]:
for play in plays:
    if play['result']['event'] in event_types:
        print(play['result']['event'])
        print(play['team']['triCode'])

Missed Shot
FLA
Shot
FLA
Shot
TBL
Blocked Shot
TBL
Shot
FLA
Shot
FLA
Blocked Shot
TBL
Shot
TBL
Blocked Shot
FLA
Shot
FLA
Shot
FLA
Missed Shot
TBL
Shot
FLA
Missed Shot
FLA
Shot
FLA
Shot
FLA
Shot
FLA
Missed Shot
FLA
Shot
FLA
Missed Shot
FLA
Blocked Shot
TBL
Shot
FLA
Missed Shot
FLA
Shot
TBL
Missed Shot
TBL
Shot
TBL
Shot
TBL
Shot
TBL
Shot
FLA
Missed Shot
TBL
Shot
TBL
Shot
FLA
Shot
FLA
Shot
TBL
Shot
FLA
Shot
FLA
Shot
FLA
Shot
TBL
Blocked Shot
TBL
Shot
FLA
Shot
FLA
Blocked Shot
TBL
Shot
FLA
Shot
FLA
Shot
FLA
Blocked Shot
TBL
Shot
FLA
Shot
TBL
Shot
TBL
Shot
TBL
Shot
TBL
Shot
FLA
Shot
FLA
Shot
FLA
Blocked Shot
TBL
Shot
TBL
Blocked Shot
TBL
Blocked Shot
FLA
Blocked Shot
FLA
Blocked Shot
TBL
Missed Shot
TBL
Shot
TBL
Shot
FLA
Missed Shot
FLA
Shot
TBL
Missed Shot
TBL
Shot
TBL
Shot
TBL
Blocked Shot
FLA
Blocked Shot
FLA
Shot
TBL
Blocked Shot
FLA
Shot
FLA
Blocked Shot
TBL
Shot
FLA
Blocked Shot
FLA
Shot
FLA
Blocked Shot
TBL
Blocked Shot
TBL
Blocked Shot
TBL
Shot
FLA
Shot
FLA
Shot
FLA
Shot
FLA
Shot
TB